# Translator for soccer data from http://www.kicker.de


In [1]:
import os
import six
from google.cloud import translate_v2 as translate
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="pmb-key.json"

In [3]:
def translate_text(target, text):
    
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    
    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    return(result["translatedText"])

In [12]:
translation_df = pd.read_csv('../data/pre_translate/' + '2018-19__match_infos_with_grades_and_summary__populated_pre_trans.csv')  

In [18]:
# drop superfluous cols and rows
translation_df = translation_df.drop(['spieltag', 'link'], axis=1).dropna()

In [19]:
translation_df.head(5)

,date,name_home_team,name_away_team,score_home,score_away,shots_home,shots_away,passes_home,passes_away,misplaced_passes_home,...,pass_accuracy_away,distance_home,distance_away,grade,summary_german,summary_english,coach_home,coach_away,interview_home,interview_away
0,"24.08.2018, 20:30",Bayern München,TSG Hoffenheim,3.0,1.0,16.0,8.0,559.0,277.0,78.0,...,75.0,115.42,119.28,"2,5",Intensives Spiel mit hohem Unterhaltungswert.,Intensives Spiel mit hohem Unterhaltungswert.,Niko Kovac,Julian Nagelsmann,Die Zuschauer haben hier ein ordentliches Spie...,Wir haben in der ersten Halbzeit nicht das Spi...
1,"25.08.2018, 15:30",Hertha BSC,1. FC Nürnberg,1.0,0.0,10.0,11.0,446.0,468.0,91.0,...,78.0,113.78,117.57,"3,5","Spannende Partie, die sich fußballerisch aller...","Spannende Partie, die sich fußballerisch aller...",Pal Dardai,Michael Köllner,"Nürnberg war eine sehr gut organisierte, cleve...",Wir wollten heute einen guten Auftakt in der e...
2,"25.08.2018, 15:30",Werder Bremen,Hannover 96,1.0,1.0,14.0,13.0,539.0,445.0,95.0,...,80.0,121.50,123.60,"3,0",Nach langem Abtasten erst im zweiten Durchgang...,Nach langem Abtasten erst im zweiten Durchgang...,Florian Kohfeldt,André Breitenreiter,Wir haben nur eine Großchance zugelassen - das...,Wir haben ein leistungsgerechtes Unentschieden...
3,"25.08.2018, 15:30",SC Freiburg,Eintracht Frankfurt,0.0,2.0,22.0,10.0,417.0,384.0,108.0,...,71.0,121.89,124.77,"3,0","Zwischendurch immer mal wieder Leerlauf, insge...","Zwischendurch immer mal wieder Leerlauf, insge...",Lars Voßler,Adolf Hütter,"Es ist schade, dass wir am Ende ohne Punkte da...",Es tut gut bei so einem heimstarken Team wie d...
4,"25.08.2018, 15:30",VfL Wolfsburg,FC Schalke 04,2.0,1.0,10.0,9.0,428.0,298.0,103.0,...,65.0,110.85,106.01,"3,0","Umkämpft, nach der Pause zunehmend intensiv, i...","Umkämpft, nach der Pause zunehmend intensiv, i...",Bruno Labbadia,Domenico Tedesco,Es war ein extrem intensives Spiel. Ich freue ...,Wir haben in den ersten 15 Minuten ein gutes S...


<div style='background:#75CF63; padding: 15px; border: thin solid black; border-radius: 5px; text-align: center'>
translate match summary

In [20]:
translation_df['summary_english'] = translation_df['summary_german'].apply(lambda x: translate_text('en', x))
display(translation_df)

,date,name_home_team,name_away_team,score_home,score_away,shots_home,shots_away,passes_home,passes_away,misplaced_passes_home,...,pass_accuracy_away,distance_home,distance_away,grade,summary_german,summary_english,coach_home,coach_away,interview_home,interview_away
0,"24.08.2018, 20:30",Bayern München,TSG Hoffenheim,3.0,1.0,16.0,8.0,559.0,277.0,78.0,...,75.0,115.42,119.28,"2,5",Intensives Spiel mit hohem Unterhaltungswert.,Intense game with a high entertainment value.,Niko Kovac,Julian Nagelsmann,Die Zuschauer haben hier ein ordentliches Spie...,Wir haben in der ersten Halbzeit nicht das Spi...
1,"25.08.2018, 15:30",Hertha BSC,1. FC Nürnberg,1.0,0.0,10.0,11.0,446.0,468.0,91.0,...,78.0,113.78,117.57,"3,5","Spannende Partie, die sich fußballerisch aller...","Exciting game, which, however, was at a medioc...",Pal Dardai,Michael Köllner,"Nürnberg war eine sehr gut organisierte, cleve...",Wir wollten heute einen guten Auftakt in der e...
2,"25.08.2018, 15:30",Werder Bremen,Hannover 96,1.0,1.0,14.0,13.0,539.0,445.0,95.0,...,80.0,121.50,123.60,"3,0",Nach langem Abtasten erst im zweiten Durchgang...,"After a long palpation, only in the second rou...",Florian Kohfeldt,André Breitenreiter,Wir haben nur eine Großchance zugelassen - das...,Wir haben ein leistungsgerechtes Unentschieden...
3,"25.08.2018, 15:30",SC Freiburg,Eintracht Frankfurt,0.0,2.0,22.0,10.0,417.0,384.0,108.0,...,71.0,121.89,124.77,"3,0","Zwischendurch immer mal wieder Leerlauf, insge...","Every now and then idle, but overall an entert...",Lars Voßler,Adolf Hütter,"Es ist schade, dass wir am Ende ohne Punkte da...",Es tut gut bei so einem heimstarken Team wie d...
4,"25.08.2018, 15:30",VfL Wolfsburg,FC Schalke 04,2.0,1.0,10.0,9.0,428.0,298.0,103.0,...,65.0,110.85,106.01,"3,0","Umkämpft, nach der Pause zunehmend intensiv, i...","Contested, increasingly intense after the brea...",Bruno Labbadia,Domenico Tedesco,Es war ein extrem intensives Spiel. Ich freue ...,Wir haben in den ersten 15 Minuten ein gutes S...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,"18.05.2019, 15:30",Werder Bremen,RB Leipzig,2.0,1.0,6.0,10.0,446.0,452.0,90.0,...,81.0,111.87,110.22,"3,5",Der finale Unterhaltungswert entschädigte für ...,The final entertainment value made up for the ...,Florian Kohfeldt,Ralf Rangnick,Natürlich ist da ein Hauch von Enttäuschung. A...,Wir hatten diese Partie 60 bis 70 Minuten im G...
302,"18.05.2019, 15:30",SC Freiburg,1. FC Nürnberg,5.0,1.0,18.0,14.0,677.0,421.0,57.0,...,85.0,117.52,113.73,"2,5","Eine sehr einseitige Partie, die durch die Fre...","A very one-sided game, which thanks to the Fre...",Christian Streich,Boris Schommers,Wir hatten unsere Durststrecke in den vergange...,Ich muss mich bei unseren Fans entschuldigen. ...
303,"18.05.2019, 15:30",1. FSV Mainz 05,TSG Hoffenheim,4.0,2.0,22.0,19.0,458.0,350.0,61.0,...,79.0,116.74,111.92,"2,0","Sehr attraktiv, viele Torraumszenen, aber auch...","Very attractive, many goal area scenes, but al...",Sandro Schwarz,Julian Nagelsmann,Die Gelb-Rote Karte kam für uns letztlich zu e...,Es ist ein Spiegelbild der gesamten Saison. Wi...
304,"18.05.2019, 15:30",VfL Wolfsburg,FC Augsburg,8.0,1.0,18.0,12.0,452.0,372.0,90.0,...,74.0,112.73,109.53,"2,0",Wolfsburger Spektakel gegen einen Gegner ohne ...,Wolfsburg spectacle against an opponent withou...,Bruno Labbadia,Martin Schmidt,"Man kann sich gar nicht erträumen, dass man so...","Wir waren mit dem letzten Aufgebot hier, klar,..."


<div style='background:#75CF63; padding: 15px; border: thin solid black; border-radius: 5px; text-align: center'>
translate home coach interviews

In [21]:
translation_df['interview_home_english'] = translation_df['interview_home'].apply(lambda x: translate_text('en', x))
display(translation_df)

,date,name_home_team,name_away_team,score_home,score_away,shots_home,shots_away,passes_home,passes_away,misplaced_passes_home,...,distance_home,distance_away,grade,summary_german,summary_english,coach_home,coach_away,interview_home,interview_away,interview_home_english
0,"24.08.2018, 20:30",Bayern München,TSG Hoffenheim,3.0,1.0,16.0,8.0,559.0,277.0,78.0,...,115.42,119.28,"2,5",Intensives Spiel mit hohem Unterhaltungswert.,Intense game with a high entertainment value.,Niko Kovac,Julian Nagelsmann,Die Zuschauer haben hier ein ordentliches Spie...,Wir haben in der ersten Halbzeit nicht das Spi...,"The spectators saw a decent game here, and our..."
1,"25.08.2018, 15:30",Hertha BSC,1. FC Nürnberg,1.0,0.0,10.0,11.0,446.0,468.0,91.0,...,113.78,117.57,"3,5","Spannende Partie, die sich fußballerisch aller...","Exciting game, which, however, was at a medioc...",Pal Dardai,Michael Köllner,"Nürnberg war eine sehr gut organisierte, cleve...",Wir wollten heute einen guten Auftakt in der e...,"Nürnberg was a very well organized, clever tea..."
2,"25.08.2018, 15:30",Werder Bremen,Hannover 96,1.0,1.0,14.0,13.0,539.0,445.0,95.0,...,121.50,123.60,"3,0",Nach langem Abtasten erst im zweiten Durchgang...,"After a long palpation, only in the second rou...",Florian Kohfeldt,André Breitenreiter,Wir haben nur eine Großchance zugelassen - das...,Wir haben ein leistungsgerechtes Unentschieden...,We only allowed one big chance - that was the ...
3,"25.08.2018, 15:30",SC Freiburg,Eintracht Frankfurt,0.0,2.0,22.0,10.0,417.0,384.0,108.0,...,121.89,124.77,"3,0","Zwischendurch immer mal wieder Leerlauf, insge...","Every now and then idle, but overall an entert...",Lars Voßler,Adolf Hütter,"Es ist schade, dass wir am Ende ohne Punkte da...",Es tut gut bei so einem heimstarken Team wie d...,It&#39;s a shame that we ended up with no poin...
4,"25.08.2018, 15:30",VfL Wolfsburg,FC Schalke 04,2.0,1.0,10.0,9.0,428.0,298.0,103.0,...,110.85,106.01,"3,0","Umkämpft, nach der Pause zunehmend intensiv, i...","Contested, increasingly intense after the brea...",Bruno Labbadia,Domenico Tedesco,Es war ein extrem intensives Spiel. Ich freue ...,Wir haben in den ersten 15 Minuten ein gutes S...,It was an extremely intense game. I&#39;m very...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,"18.05.2019, 15:30",Werder Bremen,RB Leipzig,2.0,1.0,6.0,10.0,446.0,452.0,90.0,...,111.87,110.22,"3,5",Der finale Unterhaltungswert entschädigte für ...,The final entertainment value made up for the ...,Florian Kohfeldt,Ralf Rangnick,Natürlich ist da ein Hauch von Enttäuschung. A...,Wir hatten diese Partie 60 bis 70 Minuten im G...,"There is, of course, a hint of disappointment...."
302,"18.05.2019, 15:30",SC Freiburg,1. FC Nürnberg,5.0,1.0,18.0,14.0,677.0,421.0,57.0,...,117.52,113.73,"2,5","Eine sehr einseitige Partie, die durch die Fre...","A very one-sided game, which thanks to the Fre...",Christian Streich,Boris Schommers,Wir hatten unsere Durststrecke in den vergange...,Ich muss mich bei unseren Fans entschuldigen. ...,We had our losing streak in the past few games...
303,"18.05.2019, 15:30",1. FSV Mainz 05,TSG Hoffenheim,4.0,2.0,22.0,19.0,458.0,350.0,61.0,...,116.74,111.92,"2,0","Sehr attraktiv, viele Torraumszenen, aber auch...","Very attractive, many goal area scenes, but al...",Sandro Schwarz,Julian Nagelsmann,Die Gelb-Rote Karte kam für uns letztlich zu e...,Es ist ein Spiegelbild der gesamten Saison. Wi...,The second yellow card came at a good time for...
304,"18.05.2019, 15:30",VfL Wolfsburg,FC Augsburg,8.0,1.0,18.0,12.0,452.0,372.0,90.0,...,112.73,109.53,"2,0",Wolfsburger Spektakel gegen einen Gegner ohne ...,Wolfsburg spectacle against an opponent withou...,Bruno Labbadia,Martin Schmidt,"Man kann sich gar nicht erträumen, dass man so...","Wir waren mit dem letzten Aufgebot hier, klar,...",You can&#39;t even dream of having a final lik...


<div style='background:#75CF63; padding: 15px; border: thin solid black; border-radius: 5px; text-align: center'>
translate away coach interviews

In [22]:
translation_df['interview_away_english'] = translation_df['interview_away'].apply(lambda x: translate_text('en', x))
display(translation_df)

,date,name_home_team,name_away_team,score_home,score_away,shots_home,shots_away,passes_home,passes_away,misplaced_passes_home,...,distance_away,grade,summary_german,summary_english,coach_home,coach_away,interview_home,interview_away,interview_home_english,interview_away_english
0,"24.08.2018, 20:30",Bayern München,TSG Hoffenheim,3.0,1.0,16.0,8.0,559.0,277.0,78.0,...,119.28,"2,5",Intensives Spiel mit hohem Unterhaltungswert.,Intense game with a high entertainment value.,Niko Kovac,Julian Nagelsmann,Die Zuschauer haben hier ein ordentliches Spie...,Wir haben in der ersten Halbzeit nicht das Spi...,"The spectators saw a decent game here, and our...",We didn&#39;t play the game in the first half ...
1,"25.08.2018, 15:30",Hertha BSC,1. FC Nürnberg,1.0,0.0,10.0,11.0,446.0,468.0,91.0,...,117.57,"3,5","Spannende Partie, die sich fußballerisch aller...","Exciting game, which, however, was at a medioc...",Pal Dardai,Michael Köllner,"Nürnberg war eine sehr gut organisierte, cleve...",Wir wollten heute einen guten Auftakt in der e...,"Nürnberg was a very well organized, clever tea...",We wanted to get off to a good start in the fi...
2,"25.08.2018, 15:30",Werder Bremen,Hannover 96,1.0,1.0,14.0,13.0,539.0,445.0,95.0,...,123.60,"3,0",Nach langem Abtasten erst im zweiten Durchgang...,"After a long palpation, only in the second rou...",Florian Kohfeldt,André Breitenreiter,Wir haben nur eine Großchance zugelassen - das...,Wir haben ein leistungsgerechtes Unentschieden...,We only allowed one big chance - that was the ...,We saw a performance-based draw. We can be sat...
3,"25.08.2018, 15:30",SC Freiburg,Eintracht Frankfurt,0.0,2.0,22.0,10.0,417.0,384.0,108.0,...,124.77,"3,0","Zwischendurch immer mal wieder Leerlauf, insge...","Every now and then idle, but overall an entert...",Lars Voßler,Adolf Hütter,"Es ist schade, dass wir am Ende ohne Punkte da...",Es tut gut bei so einem heimstarken Team wie d...,It&#39;s a shame that we ended up with no poin...,"With a strong home team like SC Freiburg, it&#..."
4,"25.08.2018, 15:30",VfL Wolfsburg,FC Schalke 04,2.0,1.0,10.0,9.0,428.0,298.0,103.0,...,106.01,"3,0","Umkämpft, nach der Pause zunehmend intensiv, i...","Contested, increasingly intense after the brea...",Bruno Labbadia,Domenico Tedesco,Es war ein extrem intensives Spiel. Ich freue ...,Wir haben in den ersten 15 Minuten ein gutes S...,It was an extremely intense game. I&#39;m very...,"We played a good game in the first 15 minutes,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,"18.05.2019, 15:30",Werder Bremen,RB Leipzig,2.0,1.0,6.0,10.0,446.0,452.0,90.0,...,110.22,"3,5",Der finale Unterhaltungswert entschädigte für ...,The final entertainment value made up for the ...,Florian Kohfeldt,Ralf Rangnick,Natürlich ist da ein Hauch von Enttäuschung. A...,Wir hatten diese Partie 60 bis 70 Minuten im G...,"There is, of course, a hint of disappointment....",We had this game under control for 60 to 70 mi...
302,"18.05.2019, 15:30",SC Freiburg,1. FC Nürnberg,5.0,1.0,18.0,14.0,677.0,421.0,57.0,...,113.73,"2,5","Eine sehr einseitige Partie, die durch die Fre...","A very one-sided game, which thanks to the Fre...",Christian Streich,Boris Schommers,Wir hatten unsere Durststrecke in den vergange...,Ich muss mich bei unseren Fans entschuldigen. ...,We had our losing streak in the past few games...,I have to apologize to our fans. We wanted to ...
303,"18.05.2019, 15:30",1. FSV Mainz 05,TSG Hoffenheim,4.0,2.0,22.0,19.0,458.0,350.0,61.0,...,111.92,"2,0","Sehr attraktiv, viele Torraumszenen, aber auch...","Very attractive, many goal area scenes, but al...",Sandro Schwarz,Julian Nagelsmann,Die Gelb-Rote Karte kam für uns letztlich zu e...,Es ist ein Spiegelbild der gesamten Saison. Wi...,The second yellow card came at a good time for...,It&#39;s a reflection of the entire season. We...
304,"18.05.2019, 15:30",VfL Wolfsburg,FC Augsburg,8.0,1.0,18.0,12.0,452.0,372.0,90.0,...,109.53,"2,0",Wolfsburger Spektakel gegen einen

<div style='background:#75CF63; padding: 15px; border: thin solid black; border-radius: 5px; text-align: center'>
additional cleanup

In [23]:
cleanup_df = translation_df.copy()
cleanup_df['interview_home_english'] = cleanup_df['interview_home_english'].apply(lambda x: x.replace('&#39;', '\''))
cleanup_df['interview_away_english'] = cleanup_df['interview_away_english'].apply(lambda x: x.replace('&#39;', '\''))
cleanup_df['summary_english'] = cleanup_df['summary_english'].apply(lambda x: x.replace('&#39;', '\''))

In [24]:
cleanup_df.head(50)

,date,name_home_team,name_away_team,score_home,score_away,shots_home,shots_away,passes_home,passes_away,misplaced_passes_home,...,distance_away,grade,summary_german,summary_english,coach_home,coach_away,interview_home,interview_away,interview_home_english,interview_away_english
0,"24.08.2018, 20:30",Bayern München,TSG Hoffenheim,3.0,1.0,16.0,8.0,559.0,277.0,78.0,...,119.28,"2,5",Intensives Spiel mit hohem Unterhaltungswert.,Intense game with a high entertainment value.,Niko Kovac,Julian Nagelsmann,Die Zuschauer haben hier ein ordentliches Spie...,Wir haben in der ersten Halbzeit nicht das Spi...,"The spectators saw a decent game here, and our...",We didn't play the game in the first half that...
1,"25.08.2018, 15:30",Hertha BSC,1. FC Nürnberg,1.0,0.0,10.0,11.0,446.0,468.0,91.0,...,117.57,"3,5","Spannende Partie, die sich fußballerisch aller...","Exciting game, which, however, was at a medioc...",Pal Dardai,Michael Köllner,"Nürnberg war eine sehr gut organisierte, cleve...",Wir wollten heute einen guten Auftakt in der e...,"Nürnberg was a very well organized, clever tea...",We wanted to get off to a good start in the fi...
2,"25.08.2018, 15:30",Werder Bremen,Hannover 96,1.0,1.0,14.0,13.0,539.0,445.0,95.0,...,123.60,"3,0",Nach langem Abtasten erst im zweiten Durchgang...,"After a long palpation, only in the second rou...",Florian Kohfeldt,André Breitenreiter,Wir haben nur eine Großchance zugelassen - das...,Wir haben ein leistungsgerechtes Unentschieden...,We only allowed one big chance - that was the ...,We saw a performance-based draw. We can be sat...
3,"25.08.2018, 15:30",SC Freiburg,Eintracht Frankfurt,0.0,2.0,22.0,10.0,417.0,384.0,108.0,...,124.77,"3,0","Zwischendurch immer mal wieder Leerlauf, insge...","Every now and then idle, but overall an entert...",Lars Voßler,Adolf Hütter,"Es ist schade, dass wir am Ende ohne Punkte da...",Es tut gut bei so einem heimstarken Team wie d...,It's a shame that we ended up with no points. ...,"With a strong home team like SC Freiburg, it's..."
4,"25.08.2018, 15:30",VfL Wolfsburg,FC Schalke 04,2.0,1.0,10.0,9.0,428.0,298.0,103.0,...,106.01,"3,0","Umkämpft, nach der Pause zunehmend intensiv, i...","Contested, increasingly intense after the brea...",Bruno Labbadia,Domenico Tedesco,Es war ein extrem intensives Spiel. Ich freue ...,Wir haben in den ersten 15 Minuten ein gutes S...,It was an extremely intense game. I'm very hap...,"We played a good game in the first 15 minutes,..."
5,"25.08.2018, 15:30",Fortuna Düsseldorf,FC Augsburg,1.0,2.0,8.0,14.0,447.0,422.0,99.0,...,116.98,"4,0",Intensive Partie mit vielen kämpferischen Elem...,"Intense game with many combative elements, but...",Friedhelm Funkel,Manuel Baum,Wir haben am Anfang große Schwierigkeiten geha...,Wir sind richtig gut in die Partie gekommen un...,"We had great difficulties at the beginning, bu...",We started the game really well and created go...
6,"25.08.2018, 18:30",Bor. Mönchengladbach,Bayer 04 Leverkusen,2.0,0.0,18.0,14.0,478.0,497.0,69.0,...,119.00,"2,0",Ein in vielen Phasen hochklassiges und tempore...,A high-class and fast-paced game in many phase...,Dieter Hecking,Heiko Herrlich,"Ich bin froh, dass wir so gestartet sind. Wir ...",Wir haben es in der ersten Halbzeit gut gemach...,I'm glad we started like this. We attacked ear...,"We did well in the first half, but we didn't t..."
7,"26.08.2018, 15:30",1. FSV Mainz 05,VfB Stuttgart,1.0,0.0,10.0,12.0,403.0,453.0,102.0,...,110.96,"4,5","Viel Stückwerk, einige Fehler und wenige Torra...","Much patchwork, a few mistakes and a few goal ...",Sandro Schwarz,Tayfun Korkut,Hintenraus haben wir etwas mehr Entschlossenhe...,Wir sind gut in die erste Halbzeit gekommen. W...,"In the back, we showed a little more determina...",We got into the first half well. We knew it wa...
8,"26.08.2018, 18:00",Borussia Dortmund,RB Leipzig,4.0,1.0,10.0,11.0,383.0,437.0,88.0,...,115.08,"2,0","Insbesondere vor der Pause sehr hohes Tempo, i...","Especially before the break, very fast pace, o...",Lucien F

In [25]:
cleanup_df.to_csv(r'../data/translated/2018-19__match_infos_with_grades_and_summary_translated_cleaned.csv', index = False, encoding='utf-8-sig')